# A3: A*, IDS, and Effective Branching Factor

*Luke Burford*

## Iterative Deepening Functions: ##

**iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):**
It performs an iterative deepening search, beginning at the startState provided as an argument, and attempts to reach goalState. It also takes functions actionsF and takeActionF as arguments, which are expected to provide the potential actions given a particular state (actionsF), and actually take the action, returning the new state (takeActionF). MaxDepth is the depth to which the recursion should go; if the program reaches this limit before the goalState is reached, it will return 'cutoff'.

In [236]:
nNodesGlobal = 0

In [237]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    global nNodesGlobal 
    nNodesGlobal = 0
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            result.insert(0, startState)
            #Add startState to front of solution path, in result, returned by depthLimitedSearch       
            return result
    return 'cutoff'

**depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):**
This function is essentially a helper method for iterativeDeepeningSearch() above. This function will recursively perform a depth-first search until it reaches depthLimit, which is provided to it as an argument. The arguments are essentially the same as for the function above, excepting state, which is no-longer required to be the starting state, instead it represents the "state" of the search at the time the function is called.  

This function is also responsible for generating and taking actions, using the functions provided as an argument (for further detail see above).

In [238]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    global nNodesGlobal
    if state == goalState:
        return []
    if depthLimit is 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        nNodesGlobal += 1
        childState = takeActionF(state, action)[0]
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result is 'cutoff':
            cutOff = True
        elif result is not 'failure':
            result.insert(0, childState)
            return result
    if cutOff:
        return 'cutoff'
    else:
        return 'failure'

## A* Functions:

**Class Node:**

In [239]:
class Node:
    def __init__(self, state, parent, f,g,h):
        self.state = state
        self.parent = parent
        self.f = f
        self.g = g
        self.h = h
        global nNodesGlobal
        nNodesGlobal += 1

**def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):**

In [240]:
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    global nNodesGlobal
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    nNodesGlobal = 0
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float(50))

**def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):**

In [241]:
def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        children.sort(key = lambda n: n.f)
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        alternativef = children[1].f if len(children) > 1 else float('inf')
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":                       
            result.insert(0,parentNode.state)            
            return (result, bestChild.f)

**huristic functions**

In [242]:
def h1_8p(state, goal):
    return 0

def h2_8p(state, goal):
    stateBlank = findBlank_8p(state)
    goalBlank = findBlank_8p(goal)
    x = abs(stateBlank[0] - goalBlank[0])
    y = abs(stateBlank[1] - goalBlank[1]) 
    return x + y

def h3_8p(state, goal):
    rstateBlank = findBlank_8p(state)
    goalBlank = findBlank_8p(goal)
    x = abs(stateBlank[0] - goalBlank[0])
    y = abs(stateBlank[1] - goalBlank[1]) 
    return 2(x + y)


## 8-Puzzle Helper Methods##

**printPath_8p(startState, goalState, path):**
This function takes in a starting state, the goal state, and the path returned by the iterative deepening search. It will print the results of the search if successful.

In [243]:
def printPath_8p(startState, goalState, path):
    printState_8p(startState)
    for state in path:
        printState_8p(state)
    printState_8p(goalState)

**printState_8p(state):**
This function serves as a helper method for printPath_8p. It takes a single instance (state) of the 8-puzzle, and prints it.

In [244]:
def printState_8p(state):
    tempState = state
    if type(state) is tuple:
        tempState = state[0]
    for i in range(0,3):
        if i is not 0:
            print("")
        for j in range(0,3):
            tempVal = tempState[3*i+j]
            if tempVal is 0:
                tempVal = '_'
            print(str(tempVal), end=" ")
    print("")

**takeActionF_8p(state, action):**
    This function performs the action provided as an argument to a (deep) copy of the given state, and returns this new state. In the 8-puzzle example, this would mean moving the blank piece in an acceptable direction (note that it does not perform any checking on whether an action is allowable; that is assumed to have already been done).

In [245]:
import copy
def takeActionF_8p(state, action):
    row, col = findBlank_8p(state)
    newState = copy.deepcopy(state)
    temp = -1
    if action[0] is 'left':
        temp = newState[3*row+(col-1)]
        newState[3*row+(col-1)] = 0    
    elif action[0] is 'right':
        temp = newState[3*row+(col+1)]
        newState[3*row+(col+1)] = 0
    elif action[0] is 'up':
        temp = newState[3*(row-1) + col]
        newState[3*(row-1) + col] = 0
    elif action[0] is 'down':
        temp = newState[3*(row+1) + col]
        newState[3*(row+1) + col] = 0
    else:
        print("error in takeActionF_8p")
        return -1
    newState[3*row+col] = temp
    #return newState
    return (newState, action[1])

**actionsF_8p(state):**
This function will return a generator consisting of available moves depending on the state it is provided as an argument. The possible moves are 'left', 'right', 'up', and 'down', in that order. It does perform boundary-checking, so it will only return possible moves; i.e. if the blank is at the top border of the puzzle, this function will not return 'up' as an allowable move (as it would be impossible to move further up). 

In [246]:
def actionsF_8p(state):
    row, col = findBlank_8p(state)
    #check if we can move left, right, up, or down (in that order)
    if col > 0:
        yield ('left', 1)
    if col < 2:
        yield ('right', 1)
    if row > 0:
        yield ('up', 1)
    if row < 2:
        yield ('down', 1)

**findBlank_8p(state):**
This function will locate the blank within the 8-puzzle, and return its corresponding row, column in the form of a tuple (row, col).

In [247]:
def findBlank_8p(state):
    for i in range(0,3):
        for j in range(0,3):
            if state[i*3+j] is 0:
                return (i,j)
    print("error occured in findBlank_8p")
    return -1

## ebf Functions

In [248]:
def ebf(nNodes, depth, precision = 0.01):
    if nNodes is 1 and depth is 0:
        return 0.0
    else:
        max = nNodes
        min = 1.0
        return ebfHelper(nNodes, depth, min, max, precision)

In [249]:
def ebfHelper(nNodes, depth, min, max, precision):
    mid = min + (max - min)/2
    guessNNodes = (1 - mid**(depth+1)) / (1 - mid)
    if abs(nNodes - guessNNodes) < precision:
        return mid
    elif guessNNodes > nNodes:
        return ebfHelper(nNodes, depth, min, mid, precision)
    else:
        return ebfHelper(nNodes, depth, mid, max, precision)

**Testing EBF Functionality:**

In [250]:
ebf(10, 3)

1.661376953125

In [251]:
ebf(1, 0)

0.0

In [252]:
ebf(2, 1)

1.0078125

In [253]:
ebf(2, 1, precision=0.000001)

1.0000009536743164

In [254]:
ebf(200000, 5)

11.275596931956898

## Iterative Deepening / 8-Puzzle Method Testing:

In [255]:
startState = [1, 0, 3, 4, 2, 5, 6, 7, 8]

In [256]:
printState_8p(startState)  

1 _ 3 
4 2 5 
6 7 8 


In [257]:
findBlank_8p(startState)

(0, 1)

In [258]:
test = actionsF_8p(startState)
for action in test:
    print(str(action))

('left', 1)
('right', 1)
('down', 1)


In [259]:
takeActionF_8p(startState, ('down', 1))

([1, 2, 3, 4, 0, 5, 6, 7, 8], 1)

In [260]:
printState_8p(takeActionF_8p(startState, ('down', 1)))

1 2 3 
4 _ 5 
6 7 8 


In [261]:
goalState = takeActionF_8p(startState, ('down', 1))[0]

In [262]:
newState = takeActionF_8p(startState, ('down', 1))[0]

In [263]:
newState == goalState

True

In [264]:
goalState

[1, 2, 3, 4, 0, 5, 6, 7, 8]

In [265]:
startState

[1, 0, 3, 4, 2, 5, 6, 7, 8]

In [266]:
path = depthLimitedSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

[[0, 1, 3, 4, 2, 5, 6, 7, 8],
 [1, 0, 3, 4, 2, 5, 6, 7, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

So far, we have only been testing basic functionality of various functions. Now, we move on to performing full searches:

In [267]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 3)
path

'cutoff'

In [268]:
startState = [4, 7, 2, 1, 6, 5, 0, 3, 8]
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 5)
path

'cutoff'

Both of these returned 'cutoff', which is as expected; 3 and 5 maxDepth values are not large enough to expect a different result. So, we will try 20 with this next test:

In [269]:
goalState = [1, 2, 3, 4, 0, 5, 6, 7, 8]

In [270]:
startState = [2, 7, 0, 1, 5, 3, 4, 6, 8]

In [271]:
path = iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 20)
path

[[2, 7, 0, 1, 5, 3, 4, 6, 8],
 [2, 7, 3, 1, 5, 0, 4, 6, 8],
 [2, 7, 3, 1, 0, 5, 4, 6, 8],
 [2, 0, 3, 1, 7, 5, 4, 6, 8],
 [0, 2, 3, 1, 7, 5, 4, 6, 8],
 [1, 2, 3, 0, 7, 5, 4, 6, 8],
 [1, 2, 3, 4, 7, 5, 0, 6, 8],
 [1, 2, 3, 4, 7, 5, 6, 0, 8],
 [1, 2, 3, 4, 0, 5, 6, 7, 8]]

Let's print out the state sequence in a readable form.

In [272]:
for p in path:
    printState_8p(p)
    print()

2 7 _ 
1 5 3 
4 6 8 

2 7 3 
1 5 _ 
4 6 8 

2 7 3 
1 _ 5 
4 6 8 

2 _ 3 
1 7 5 
4 6 8 

_ 2 3 
1 7 5 
4 6 8 

1 2 3 
_ 7 5 
4 6 8 

1 2 3 
4 7 5 
_ 6 8 

1 2 3 
4 7 5 
6 _ 8 

1 2 3 
4 _ 5 
6 7 8 



## Experiment Functions

In [273]:
def goalTestF_8p(state, goal):
    if state == goal:
        return True
    return False

In [274]:
def runExperiment(goalState1, goalState2, goalState3, hFList):
    global nNodesGlobal
    print(goalState1,"\t",goalState2,"\t",goalState3)
    print("Algorithm   Depth  Nodes  EBF\t Depth  Nodes  EBF\t\t Depth  Nodes  EBF")
    startState = [1,2,3,4,0,5,6,7,8]
    iDSSol = []
    iDSDepth = []
    aStarSol = []
    aStarDepth = []
    iDSSol.append(iterativeDeepeningSearch(startState,goalState1, actionsF_8p, takeActionF_8p,15))
    t = (len(iDSSol[-1])-1,nNodesGlobal)
    iDSDepth.append(t)
    iDSSol.append(iterativeDeepeningSearch(startState,goalState2, actionsF_8p, takeActionF_8p,15))
    t = (len(iDSSol[-1])-1,nNodesGlobal)
    iDSDepth.append(t)
    iDSSol.append(iterativeDeepeningSearch(startState,goalState3, actionsF_8p, takeActionF_8p,15))
    t = (len(iDSSol[-1])-1,nNodesGlobal)
    iDSDepth.append(t)
    
    for hF in hFList:
        aStarSol.append(aStarSearch(startState,actionsF_8p, takeActionF_8p, lambda s: goalTestF_8p(s, goalState1), lambda v: hF(v, goalState1))[0])
        t = (len(aStarSol[-1])-1,nNodesGlobal)
        aStarDepth.append(t)
        
        aStarSol.append(aStarSearch(startState,actionsF_8p, takeActionF_8p, lambda s: goalTestF_8p(s, goalState2), lambda v: hF(v, goalState1))[0])
        t = (len(aStarSol[-1])-1,nNodesGlobal)
        aStarDepth.append(t)
        
        aStarSol.append(aStarSearch(startState,actionsF_8p, takeActionF_8p, lambda s: goalTestF_8p(s, goalState3), lambda v: hF(v, goalState1))[0])
        t = (len(aStarSol[-1])-1,nNodesGlobal)
        aStarDepth.append(t)
    for sol in iDSSol:
        print("\t   ", end = "")
        print(str(sol[-1]), end ="   ")
    print("Algorithm\tDepth\tNodes\t  EBF\t\tDepth\tNodes\t  EBF\t\tDepth\tNodes\t  EBF")
    print("IDS\t\t  " + str(iDSDepth[0][0]) + "  \t " + str(iDSDepth[0][1]) +
         "\t" + (ebf(iDSDepth[0][1], iDSDepth[0][0])), end = "\t\t")
    print(str(iDSDepth[1][0]) + "  \t " + str(iDSDepth[1][1]) +
         "\t" + (ebf(iDSDepth[1][1], iDSDepth[1][0])), end = "\t\t")
    print(str(iDSDepth[2][0]) + "  \t " + str(iDSDepth[2][1]) +
         "\t" + (ebf(iDSDepth[2][1], iDSDepth[2][0])))

    for i in range(len(hFlist)):
        print("A*h" + str(i+1) + "\t\t" + str(aStarDepth[0][0]) + "\t" + str(aStarDepth[0][1]) + 
             "\t" + (ebf(aStarDepth[(3*i)][1], iStarDepth[3*i][0])), end = "\t\t")
        
        print(str(aStarDepth[3*i+1][0]) + "\t" + str(aStarDepth[3*i+1][1]) + 
             "\t" + (ebf(aStarDepth[(3*i+1)][1], iStarDepth[3*i+1][0])), end = "\t\t")
        
        print(str(aStarDepth[3*i+2][0]) + "\t" + str(aStarDepth[3*i+2][1]) + 
             "\t" + (ebf(aStarDepth[(3*i+1)][1], iStarDepth[3*i+2][0])))

## Comparing A* and Iterative Deepening on 8-Puzzle

In [275]:
goalState1 = [1,2,3,4,0,5,6,7,8]
goalState2 = [1,2,3,4,5,8,6,0,7]
goalState3 = [1,0,3,4,5,8,2,6,7]

In [276]:
hFList1 = [h1_8p,h2_8p,h3_8p]
solution = runExperiment(goalState1, goalState2, goalState3, hFList1)

[1, 2, 3, 4, 0, 5, 6, 7, 8] 	 [1, 2, 3, 4, 5, 8, 6, 0, 7] 	 [1, 0, 3, 4, 5, 8, 2, 6, 7]
Algorithm   Depth  Nodes  EBF	 Depth  Nodes  EBF		 Depth  Nodes  EBF


TypeError: __init__() missing 1 required positional argument: 'parent'